In [4]:
import requests
import pandas as pd
import time
from api import API_KEY

In [9]:
API_KEY = API_KEY
search_query = "data science"

search_url = "https://www.googleapis.com/youtube/v3/search"
params = {
    "part": "snippet",
    "q": search_query,
    "type": "video",
    "maxResults": 10,
    "key": API_KEY}

response = requests.get(search_url, params=params)
results = response.json()

videos = []
for item in results["items"]:
    videos.append({
        "video_id": item["id"]["videoId"],
        "title": item["snippet"]["title"],
        "channel": item["snippet"]["channelTitle"],
        "published": item["snippet"]["publishedAt"]})

df = pd.DataFrame(videos)
df

{'error': {'code': 403, 'message': 'The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.', 'errors': [{'message': 'The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.', 'domain': 'youtube.quota', 'reason': 'quotaExceeded'}]}}


KeyError: 'items'

In [ ]:
video_ids = df["video_id"].tolist()

video_stats_url = "https://www.googleapis.com/youtube/v3/videos"
params = {
    "part": "snippet,statistics",
    "id": ",".join(video_ids),
    "key": API_KEY}

response = requests.get(video_stats_url, params=params)
video_data = response.json()

video_stats = []
for item in video_data["items"]:
    stats = item["statistics"]
    snippet = item["snippet"]
    video_stats.append({
        "video_id": item["id"],
        "title": snippet["title"],
        "channel": snippet["channelTitle"],
        "published": snippet["publishedAt"],
        "views": int(stats.get("viewCount", 0)),
        "likes": int(stats.get("likeCount", 0)),
        "comments": int(stats.get("commentCount", 0))})

stats_df = pd.DataFrame(video_stats)
stats_df

In [ ]:
keywords = ["data science", "machine learning", "python tutorial", "artificial intelligence", "data analytics", "ai", "ml"]
max_results_per_keyword = 1000

all_video_data = []

for keyword in keywords:
    print(f"Searching for: {keyword}")
    search_url = "https://www.googleapis.com/youtube/v3/search"
    next_page_token = None
    total_collected = 0

    while total_collected < max_results_per_keyword:
        params = {
            "part": "snippet",
            "q": keyword,
            "type": "video",
            "maxResults": 50,
            "key": API_KEY}
        if next_page_token:
            params["pageToken"] = next_page_token

        response = requests.get(search_url, params=params)
        data = response.json()
        
        video_ids = [item["id"]["videoId"] for item in data.get("items", [])]
        total_collected += len(video_ids)

        # Fetch stats
        stats_url = "https://www.googleapis.com/youtube/v3/videos"
        stats_params = {
            "part": "snippet,statistics",
            "id": ",".join(video_ids),
            "key": API_KEY
        }
        stats_response = requests.get(stats_url, params=stats_params)
        stats_data = stats_response.json()

        for item in stats_data.get("items", []):
            snippet = item["snippet"]
            stats = item["statistics"]

            all_video_data.append({
                "keyword": keyword,
                "video_id": item["id"],
                "title": snippet["title"],
                "channel": snippet["channelTitle"],
                "published": snippet["publishedAt"],
                "views": int(stats.get("viewCount", 0)),
                "likes": int(stats.get("likeCount", 0)),
                "comments": int(stats.get("commentCount", 0))})

        next_page_token = data.get("nextPageToken")
        if not next_page_token:
            break

        time.sleep(1)  # To stay within quota

print(f"Total videos collected: {len(all_video_data)}")
df = pd.DataFrame(all_video_data)
df.to_csv("youtube_videos_1000.csv", index=False)
df.head()

In [ ]:
df.shape

In [ ]:
df["keyword"].value_counts()